In [ ]:
import seaborn as sns
from scipy import stats
from scipy.stats import norm
import statistics
from scipy.integrate import solve_ivp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures, SplineTransformer
from sklearn.pipeline import make_pipeline

#-------------------------------------------------------------------------#

#load in data
df = pd.read_csv('masterDf.csv')
df2 = df.fillna(0)

#---

#create scaled DataFrame where each variable has mean of 0 and standard dev of 1
scaled_df = StandardScaler().fit_transform(df2)

#view first five rows of scaled DataFrame
print(scaled_df[:5])

#---

#instantiate the k-means class, using optimal number of clusters
kmeans = KMeans(init="random", n_clusters=3, n_init=10, random_state=1)

#fit k-means algorithm to data
kmeans.fit(scaled_df)

#view cluster assignments for each observation
kmeans.labels_
#---

#initialize kmeans parameters
#kmeans_kwargs = {
#"init": "random",
#"n_init": 10,
#"random_state": 1,
#}

#create list to hold SSE values for each k
#sse = []
#for k in range(1, 11):
#    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
#    kmeans.fit(scaled_df)
#    sse.append(kmeans.inertia_)

#visualize results
#plt.plot(range(1, 11), sse)
#plt.xticks(range(1, 11))
#plt.xlabel("Number of Clusters")
#plt.ylabel("SSE")
#plt.show()

#---

#append cluster assingments to original DataFrame
df2['cluster'] = kmeans.labels_

#view updated DataFrame
print(df2)

#---

df3 = df2['cluster'].value_counts()
print(df3)

#df3.to_csv('./cluster.csv')


#PCA ******************************************************
def my_PCA(data):
    #compute the mean of each row 
    data_mean = data.mean(axis = 1) 
    #construct the average matrix 
    data_mat = data_mean * np.ones((1,data.shape[1])) 
    #subtract mean from data, b matrix centers the data at the origin for PCA functionality 
    b_mat = data - data_mat 
    
    #perform SVD decomposition on matrix 
    U, S, VT = np.linalg.svd(b_mat)
    #use the singular values from S to construct diag matrix 
    S = np.diag(S)
    #construct PCA 
    T = np.matmul(U,S)
    #returns S and T 
    return S, T

#read in ovarian cancer data from csv file 
df = pd.read_csv('masterDf.csv')

#convert the data using np.mat 
new_data = np.mat(df)

#return S, T by calling the PCA implementation function 
S, T = my_PCA(new_data)      

#plot the diagonal S values using semilogy 
plt.semilogy(np.diag(S), label = 'singular values and corresponding values')  
#format the plot with title, xlabel, ylabel, and legend 
plt.title("Singular values and their corresponding values")
plt.xlabel("r (Singular Value)")
plt.ylabel("Values of singular values")
plt.legend()
plt.show() 


#KMeans ****************************************************

from sklearn.cluster import KMeans

from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

# Isolate data 
# Define the parameter we wish to optimize
parameters = {'alpha':np.linspace(0, 1000, 50)} #start is 0, stop is 1000, steps is 50 
Lasso = linear_model.Lasso() # Define the model we wish to test
# Define a grid search that will use 5-fold cross validation
Lasso_reg = GridSearchCV(Lasso, parameters,
                         scoring = 'neg_mean_squared_error',
                         cv = 5,
                         return_train_score = True)

Lasso_reg.fit(df['rating'].values.reshape(-1,1) , df['movie'].values.reshape(-1,1) ) # Fit the grid search
print(Lasso_reg.best_estimator_) # Print best alpha parameter


# Finding regularization parameter value vs. error on test data
Lasso_meanTest = abs(Lasso_reg.cv_results_['mean_test_score'])
plt.figure()
plt.plot(parameters['alpha'], Lasso_meanTest, label = "LASSO")
plt.title('Using Lasso Linear Regression on Dataset')
plt.xlabel('Rating')
plt.ylabel('Movie')
plt.legend()
plt.show()